In [ ]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.smugplug import SmugPlugControllerConfig
from decimal import Decimal


class SmugPlugConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for SmugPlug optimization.
    """
    async def generate_config(self, trial) -> BacktestingConfig:
        # Controller configuration
        connector_name = "binance_perpetual"
        trading_pair = "WLD-USDT"
        interval = "1h"
        macd_fast = trial.suggest_int("macd_fast", 9, 59)
        macd_slow = trial.suggest_int("macd_slow", macd_fast + 10, 201)
        macd_signal = trial.suggest_int("macd_signal", macd_fast + 5, 80)
        atr_length = trial.suggest_int("atr_length", 11, 201)
        atr_multiplier = trial.suggest_float("atr_multiplier", 0.5, 1.5)
        short_ema = trial.suggest_int("short_ema", 9, 59)
        medium_ema = trial.suggest_int("medium_ema", short_ema + 10, 150)
        long_ema = trial.suggest_int("long_ema", medium_ema + 10, 201)
        # Suggest hyperparameters using the trial object
        total_amount_quote = 1000
        max_executors_per_side = trial.suggest_int("max_executors_per_side", 1, 3)
        take_profit = trial.suggest_float("take_profit", 0.04, 0.05, step=0.01)
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.05, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.01, 0.02, step=0.005)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.1, 0.3, step=0.1)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        time_limit = 60 * 60 * 24 * 2
        cooldown_time = 60 * 15

        # Create the strategy configuration
        config = SmugPlugControllerConfig(
            connector_name=connector_name,
            trading_pair=trading_pair,
            interval=interval,
            macd_fast=macd_fast,
            macd_slow=macd_slow,
            macd_signal=macd_signal,
            atr_length=atr_length,
            atr_multiplier=atr_multiplier,
            ema_short=short_ema,
            ema_medium=medium_ema,
            ema_long=long_ema,
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=TrailingStop(
                activation_price=Decimal(trailing_stop_activation_price),
                trailing_delta=Decimal(trailing_stop_trailing_delta),
            ),
            time_limit=time_limit,
            max_executors_per_side=max_executors_per_side,
            cooldown_time=cooldown_time,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2024, 8, 1)
end_date = datetime.datetime(2024, 10, 1)
config_generator = SmugPlugConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer(root_path=root_path)
# await optimizer.optimize(
#     study_name="smugplug_v1_with_optimizer",
#     config_generator=config_generator,
#     n_trials=100,
# )

In [ ]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()